In [11]:
import pandas as pd
import plotly.express as px
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from dash import no_update

In [2]:
#add a dataframe
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "NYC", "MTL", "NYC"]
})

In [3]:
#add a bar graph figure
fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")


In [4]:
#create a dash app
app = dash.Dash()

#build app layout

app.layout = html.Div(children=[html.H1(children='Dashboard', 
                                       style={'textAlign': 'center'}),
                               # Create dropdown
                                dcc.Dropdown(options=[
                                    {'label': 'New York City', 'value': 'NYC'},
                                    {'label': 'Montréal', 'value': 'MTL'},
                                    {'label': 'San Francisco', 'value': 'SF'}],
                                             value='NYC' # Providing a value to dropdown
                                            ),
                                dcc.Graph(id='example-graph-2', figure=fig)
                               ])

In [10]:
# Run the application                   
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [08/Feb/2023 02:27:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2023 02:27:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2023 02:27:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2023 02:27:10] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2023 02:27:10] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Feb/2023 02:27:10] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


# Working on a real dataset now

- As a data analyst, you have been given a task to visually show the body-style and prices with respect to each drive wheel selected.

- So ideally you want to showcase this in the form of 2 interactive charts such as pie chart and bar chart on selection of drive wheel.


In [9]:
car_data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/automobileEDA.csv',
                      encoding = "ISO-8859-1")
car_data.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


In [13]:
#creating the dash app
app2 = dash.Dash()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app2.config.suppress_callback_exceptions = True

#build layout
app2.layout= html.Div(children=[html.H1('Car Automobile Components',
                                        style={'textAlign': 'center',
                                               'color': '#503D36',
                                               'font-size': 50}),
                                #adding outer division
                                html.Div(
                                    #adding first inner division for dropdown
                                    html.Div([html.H2('Drive Wheels Type:', style={'margin-right': '2em'}),
                                #adding dropdown
                                dcc.Dropdown(id = 'demo-dropdown',
                                             options=[
                                    {'label': 'Front wheel drive', 'value':'fwd'},
                                    {'label': 'Rear wheel drive', 'value':'rwd'},
                                    {'label': 'All wheel drive', 'value':'4wd'}
                                ], value='fwd'),
                                    
                                    #adding second inner division for two graphs
                                    html.Div([
                                        html.Div(dcc.Graph(id='plot1')),
                                        html.Div(dcc.Graph(id='plot2'))
                                    ], style={'display':'flex'})])
                                    
                               
                                )])

In [19]:
#creating the decorator

@app2.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children')],
              Input(component_id='demo-dropdown', component_property=''))
def df_per_drive_wheel(value):
    filtered_df = car_data[car_data['drive-wheels']==value].groupby(['drive-wheels','body-style'], as_index=False).mean()
    filtered_df = filtered_df
    fig1 = px.pie(filtered_df, values='price', names='body-style', title="Pie Chart")
    fig2 = px.bar(filtered_df, x='body-style', y='price', title='Bar Chart')

    return [dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2) ]